In [56]:
from datasets import load_dataset
#import grelu.resources
import numpy as np
import pandas as pd
import polars as pl
from sklearn.metrics import average_precision_score
from tqdm import tqdm

In [18]:
#project, model_name = "enformer", "human"
#project, model_name = "borzoi", "human_fold0"
#metadata = pd.DataFrame(
#    grelu.resources.load_model(
#        project=project, model_name=model_name
#    ).data_params['tasks']
#)
#metadata.to_csv("Enformer_metadata.csv", index=False)
metadata = pd.read_csv("Enformer_metadata.csv")
metadata

,name,file,clip,scale,sum_stat,description,assay,sample
0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...,DNASE,cerebellum male adult (27 years) and male adul...
1,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...,DNASE,frontal cortex male adult (27 years) and male ...
2,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion,DNASE,chorion
3,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...,DNASE,Ishikawa treated with 0.02% dimethyl sulfoxide...
4,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348,DNASE,GM03348
...,...,...,...,...,...,...,...,...
5308,CNhs14239,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:epithelioid sarcoma cell line:HS-ES-2R,CAGE,epithelioid sarcoma cell line:HS-ES-2R
5309,CNhs14240,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:squamous cell lung carcinoma cell line:RE...,CAGE,squamous cell lung carcinoma cell line:RERF-LC-AI
5310,CNhs14241,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:gastric cancer cell line:GSS,CAGE,gastric cancer cell line:GSS
5311,CNhs14244,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:carcinoid cell line:NCI-H727,CAGE,carcinoid cell line:NCI-H727


In [41]:
metadata.assay.value_counts()

assay
CHIP     3991
DNASE     674
CAGE      638
ATAC       10
Name: count, dtype: int64

In [40]:
metadata["group"] = metadata.apply(lambda x: x.assay if x.assay != "CHIP" else ":".join(x.description.split(":")[:2]), axis=1)
metadata["group"].value_counts()

group
DNASE               674
CAGE                638
CHIP:H3K4me3        338
CHIP:H3K36me3       269
CHIP:H3K27me3       260
                   ... 
CHIP:eGFP-IKZF3       1
CHIP:eGFP-MZF1        1
CHIP:eGFP-ZNF213      1
CHIP:eGFP-ZNF777      1
CHIP:eGFP-ZNF660      1
Name: count, Length: 791, dtype: int64

In [4]:
dataset_path = "gonzalobenegas/siraj_gwas_highpip"


V = load_dataset(dataset_path, split="test").to_pandas()
V

,chrom,pos,ref,alt,label
0,1,3080038,T,C,True
1,1,3774964,A,G,True
2,1,6616131,C,T,False
3,1,7665224,C,A,False
4,1,8407293,G,A,False
...,...,...,...,...,...
1778,22,47019717,G,T,False
1779,22,47990921,C,T,True
1780,22,50007172,T,C,False
1781,22,50190508,G,A,True


In [5]:
features = "Enformer_L2"
df = pd.read_parquet(f"https://huggingface.co/datasets/{dataset_path}/resolve/main/features/{features}.parquet")
feature_cols = df.columns
V = pd.concat([V, df], axis=1)
V

,chrom,pos,ref,alt,label,ENCFF833POA,ENCFF110QGM,ENCFF880MKD,ENCFF463ZLQ,ENCFF890OGQ,...,CNhs14551,CNhs14618,CNhs14226,CNhs14229,CNhs14238,CNhs14239,CNhs14240,CNhs14241,CNhs14244,CNhs14245
0,1,3080038,T,C,True,0.020627,0.027453,0.022877,0.014595,0.011404,...,0.030669,0.028385,0.014161,0.009556,0.004341,0.008560,0.003383,0.008210,0.017461,0.018274
1,1,3774964,A,G,True,0.034373,0.026412,0.037801,0.026490,0.027976,...,0.059913,0.032793,0.019731,0.008741,0.034403,0.020556,0.023130,0.022730,0.060444,0.061170
2,1,6616131,C,T,False,0.013653,0.013146,0.022949,0.011773,0.012057,...,0.044309,0.031237,0.013514,0.054213,0.008707,0.007368,0.008335,0.008033,0.023703,0.020034
3,1,7665224,C,A,False,0.029997,0.034185,0.063735,0.015230,0.026252,...,0.091137,0.069207,0.024454,0.019247,0.004269,0.005681,0.003253,0.003965,0.014895,0.017959
4,1,8407293,G,A,False,0.020948,0.011623,0.015968,0.010985,0.022360,...,0.078673,0.040349,0.017659,0.014112,0.008577,0.012185,0.011752,0.010652,0.041655,0.026237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,22,47019717,G,T,False,0.010188,0.015055,0.031079,0.005793,0.014833,...,0.060567,0.021760,0.007151,0.014033,0.004122,0.001136,0.001977,0.002988,0.010180,0.010185
1779,22,47990921,C,T,True,0.042335,0.041975,0.008428,0.005667,0.005185,...,0.013600,0.011352,0.005881,0.005740,0.001590,0.001055,0.001519,0.001720,0.005846,0.003889
1780,22,50007172,T,C,False,0.030721,0.039224,0.036249,0.023633,0.029598,...,0.048981,0.033054,0.013186,0.011193,0.011780,0.009787,0.007663,0.008840,0.021838,0.019903
1781,22,50190508,G,A,True,0.185299,0.264034,0.273924,0.256359,0.342722,...,0.623208,0.439687,0.121340,0.054223,0.568436,0.201643,0.287033,0.352236,0.761115,0.776491


In [9]:
res = []
for c in tqdm(feature_cols):
    res.append([c, average_precision_score(V["label"], V[c])])
res = pd.DataFrame(res, columns=["feature", "AUPRC"]).sort_values("AUPRC", ascending=False)
res

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5313/5313 [00:09<00:00, 572.07it/s]


,feature,AUPRC
229,ENCFF083VCD,0.734845
270,ENCFF978RWQ,0.733138
22,ENCFF082SFS,0.732975
571,ENCFF130BQK,0.732596
26,ENCFF818FXA,0.731510
...,...,...
2374,ENCFF031GKU,0.523921
3720,ENCFF149ZYU,0.522652
4193,ENCFF078WGV,0.522403
2002,ENCFF331CAB,0.517747


In [14]:
res = res.merge(metadata, left_on="feature", right_on="name")
res

,feature,AUPRC,name,file,clip,scale,sum_stat,description,assay,sample
0,ENCFF083VCD,0.734845,ENCFF083VCD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:lung female embryo (76 days),DNASE,lung female embryo (76 days)
1,ENCFF978RWQ,0.733138,ENCFF978RWQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:stomach female embryo (98 days),DNASE,stomach female embryo (98 days)
2,ENCFF082SFS,0.732975,ENCFF082SFS,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:heart male adult (27 years) and male adu...,DNASE,heart male adult (27 years) and male adult (35...
3,ENCFF130BQK,0.732596,ENCFF130BQK,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:femur female embryo (98 days),DNASE,femur female embryo (98 days)
4,ENCFF818FXA,0.731510,ENCFF818FXA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:hepatocyte,DNASE,hepatocyte
...,...,...,...,...,...,...,...,...,...,...
5308,ENCFF031GKU,0.523921,ENCFF031GKU,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:EP300:suprapubic skin female adult (51 year),CHIP,EP300:suprapubic skin female adult (51 year)
5309,ENCFF149ZYU,0.522652,ENCFF149ZYU,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H2BK15ac:H9,CHIP,H2BK15ac:H9
5310,ENCFF078WGV,0.522403,ENCFF078WGV,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:EP300:esophagus muscularis mucosa female ...,CHIP,EP300:esophagus muscularis mucosa female adult...
5311,ENCFF331CAB,0.517747,ENCFF331CAB,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:EP300:subcutaneous adipose tissue male ad...,CHIP,EP300:subcutaneous adipose tissue male adult (...


In [15]:
res.groupby("assay").AUPRC.mean().sort_values(ascending=False)

assay
DNASE    0.703063
ATAC     0.692765
CAGE     0.667374
CHIP     0.647179
Name: AUPRC, dtype: float64

In [19]:
res.groupby("description").AUPRC.mean().sort_values(ascending=False)

description
DNASE:lung female embryo (76 days)                               0.734845
DNASE:stomach female embryo (98 days)                            0.733138
DNASE:heart male adult (27 years) and male adult (35 years)      0.732975
DNASE:femur female embryo (98 days)                              0.732596
DNASE:hepatocyte                                                 0.731510
                                                                   ...   
CHIP:EP300:suprapubic skin female adult (51 year)                0.523921
CHIP:H2BK15ac:H9                                                 0.522652
CHIP:EP300:esophagus muscularis mucosa female adult (51 year)    0.522403
CHIP:EP300:subcutaneous adipose tissue male adult (37 years)     0.517747
CHIP:EP300:breast epithelium male adult (54 years)               0.511211
Name: AUPRC, Length: 4925, dtype: float64

In [20]:
res.groupby("description").AUPRC.mean().sort_values(ascending=False).head(50)

description
DNASE:lung female embryo (76 days)                                                                    0.734845
DNASE:stomach female embryo (98 days)                                                                 0.733138
DNASE:heart male adult (27 years) and male adult (35 years)                                           0.732975
DNASE:femur female embryo (98 days)                                                                   0.732596
DNASE:hepatocyte                                                                                      0.731510
DNASE:spleen embryo (112 days)                                                                        0.730772
DNASE:right lung female embryo (105 days)                                                             0.730133
DNASE:NB4                                                                                             0.729599
DNASE:left lung male embryo (115 days)                                                              

In [23]:
res_chip = res[res.assay=="CHIP"]
res_chip

,feature,AUPRC,name,file,clip,scale,sum_stat,description,assay,sample
10,ENCFF579CVA,0.728819,ENCFF579CVA,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:subcutaneous abdominal adipose ti...,CHIP,H3K4me3:subcutaneous abdominal adipose tissue ...
14,ENCFF493FQG,0.728070,ENCFF493FQG,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:NB4,CHIP,H3K4me3:NB4
43,ENCFF035NGT,0.723262,ENCFF035NGT,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K27ac:right lobe of liver female adult ...,CHIP,H3K27ac:right lobe of liver female adult (53 y...
50,ENCFF870RAN,0.722389,ENCFF870RAN,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K27ac:upper lobe of left lung female ad...,CHIP,H3K27ac:upper lobe of left lung female adult (...
52,ENCFF024VVX,0.722357,ENCFF024VVX,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K27ac:adrenal gland female adult (53 ye...,CHIP,H3K27ac:adrenal gland female adult (53 years)
...,...,...,...,...,...,...,...,...,...,...
5308,ENCFF031GKU,0.523921,ENCFF031GKU,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:EP300:suprapubic skin female adult (51 year),CHIP,EP300:suprapubic skin female adult (51 year)
5309,ENCFF149ZYU,0.522652,ENCFF149ZYU,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H2BK15ac:H9,CHIP,H2BK15ac:H9
5310,ENCFF078WGV,0.522403,ENCFF078WGV,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:EP300:esophagus muscularis mucosa female ...,CHIP,EP300:esophagus muscularis mucosa female adult...
5311,ENCFF331CAB,0.517747,ENCFF331CAB,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:EP300:subcutaneous adipose tissue male ad...,CHIP,EP300:subcutaneous adipose tissue male adult (...


In [28]:
res_chip["mark"] = res_chip.description.str.split(":").str[1]
res_chip.groupby("mark").AUPRC.mean().sort_values(ascending=False)

/tmp/ipykernel_3223949/113203681.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_chip["mark"] = res_chip.description.str.split(":").str[1]


mark
RBFOX2            0.703089
3xFLAG-SAP130     0.696222
H3K27Ac           0.694529
3xFLAG-GATAD2A    0.692816
eGFP-PRDM10       0.691565
                    ...   
H2AK9ac           0.576007
H3K23me2          0.574407
H3K23ac           0.572393
3xFLAG-ZBED5      0.563453
PLRG1             0.560066
Name: AUPRC, Length: 788, dtype: float64

In [29]:
res_chip.groupby("mark").AUPRC.mean().sort_values(ascending=False).head(50)

mark
RBFOX2            0.703089
3xFLAG-SAP130     0.696222
H3K27Ac           0.694529
3xFLAG-GATAD2A    0.692816
eGFP-PRDM10       0.691565
NR2F2             0.690090
eGFP-GLIS1        0.689921
3xFLAG-ZGPAT      0.688471
eGFP-ZIC2         0.688342
eGFP-ZNF335       0.688163
RELB              0.688001
eGFP-ZNF341       0.687746
E2F6              0.686270
eGFP-NR2C2        0.686173
3xFLAG-KAT8       0.684598
3xFLAG-TEAD1      0.683932
3xFLAG-GABPB1     0.683761
HNRNPL            0.683642
eGFP-ATF1         0.683586
eGFP-VEZF1        0.683105
TBX21             0.682782
hBMAL1            0.682746
3xFLAG-SOX5       0.682499
3xFLAG-RARA       0.682466
3xFLAG-THAP11     0.682191
3xFLAG-ARID4B     0.681679
3xFLAG-ELF3       0.681007
3xFLAG-RXRB       0.680269
eGFP-ZNF600       0.680027
GABPB1            0.679494
H3K27ac           0.678991
H3K4me1           0.678871
eGFP-PBX2         0.678795
3xFLAG-HMG20A     0.678435
3xFLAG-FOXA3      0.677840
eGFP-ZBTB8A       0.677812
AGO2              0.677

In [32]:
res_chip_H3K4me3 = res_chip[res_chip.mark=="H3K4me3"]
res_chip_H3K4me3

,feature,AUPRC,name,file,clip,scale,sum_stat,description,assay,sample,mark
10,ENCFF579CVA,0.728819,ENCFF579CVA,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:subcutaneous abdominal adipose ti...,CHIP,H3K4me3:subcutaneous abdominal adipose tissue ...,H3K4me3
14,ENCFF493FQG,0.728070,ENCFF493FQG,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:NB4,CHIP,H3K4me3:NB4,H3K4me3
100,ENCFF535CDG,0.719474,ENCFF535CDG,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:subcutaneous abdominal adipose ti...,CHIP,H3K4me3:subcutaneous abdominal adipose tissue ...,H3K4me3
131,ENCFF179GFG,0.718162,ENCFF179GFG,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:subcutaneous abdominal adipose ti...,CHIP,H3K4me3:subcutaneous abdominal adipose tissue ...,H3K4me3
156,ENCFF327QYA,0.717199,ENCFF327QYA,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:stomach smooth muscle female adul...,CHIP,H3K4me3:stomach smooth muscle female adult (84...,H3K4me3
...,...,...,...,...,...,...,...,...,...,...,...
5184,ENCFF293ASZ,0.572609,ENCFF293ASZ,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:lung female adult (30 years),CHIP,H3K4me3:lung female adult (30 years),H3K4me3
5187,ENCFF015ZVX,0.571867,ENCFF015ZVX,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:neural stem progenitor cell origi...,CHIP,H3K4me3:neural stem progenitor cell originated...,H3K4me3
5214,ENCFF312ENS,0.567943,ENCFF312ENS,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:peripheral blood mononuclear cell...,CHIP,H3K4me3:peripheral blood mononuclear cell male...,H3K4me3
5248,ENCFF748XLQ,0.558304,ENCFF748XLQ,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:mesendoderm originated from H1-hESC,CHIP,H3K4me3:mesendoderm originated from H1-hESC,H3K4me3


In [26]:
res_chip.description.values.tolist()

['CHIP:H3K4me3:subcutaneous abdominal adipose tissue nuclear fraction female adult (81 year)',
 'CHIP:H3K4me3:NB4',
 'CHIP:H3K27ac:right lobe of liver female adult (53 years)',
 'CHIP:H3K27ac:upper lobe of left lung female adult (53 years)',
 'CHIP:H3K27ac:adrenal gland female adult (53 years)',
 'CHIP:H3K27ac:liver male adult (31 year)',
 'CHIP:H3K27ac:stomach female adult (53 years)',
 'CHIP:H3K27ac:liver female adult (25 years)',
 'CHIP:H3K4me1:subcutaneous abdominal adipose tissue nuclear fraction female adult (59 years)',
 'CHIP:H3K27ac:body of pancreas male adult (37 years)',
 'CHIP:H3K4me3:subcutaneous abdominal adipose tissue nuclear fraction female adult (49 years)',
 'CHIP:H3K4me1:subcutaneous abdominal adipose tissue nuclear fraction female adult (81 year)',
 'CHIP:H3K27ac:spleen female adult (30 years)',
 'CHIP:H3K4me2:skeletal muscle satellite cell female adult originated from mesodermal cell',
 'CHIP:H3K4me1:skeletal muscle tissue female adult (72 years)',
 'CHIP:H3K4me3:

## Aggregate features

In [45]:
assays = metadata.assay.unique()
assays

array(['DNASE', 'ATAC', 'CHIP', 'CAGE'], dtype=object)

In [47]:
metadata

,name,file,clip,scale,sum_stat,description,assay,sample,group
0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...,DNASE,cerebellum male adult (27 years) and male adul...,DNASE
1,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...,DNASE,frontal cortex male adult (27 years) and male ...,DNASE
2,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion,DNASE,chorion,DNASE
3,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...,DNASE,Ishikawa treated with 0.02% dimethyl sulfoxide...,DNASE
4,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348,DNASE,GM03348,DNASE
...,...,...,...,...,...,...,...,...,...
5308,CNhs14239,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:epithelioid sarcoma cell line:HS-ES-2R,CAGE,epithelioid sarcoma cell line:HS-ES-2R,CAGE
5309,CNhs14240,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:squamous cell lung carcinoma cell line:RE...,CAGE,squamous cell lung carcinoma cell line:RERF-LC-AI,CAGE
5310,CNhs14241,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:gastric cancer cell line:GSS,CAGE,gastric cancer cell line:GSS,CAGE
5311,CNhs14244,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:carcinoid cell line:NCI-H727,CAGE,carcinoid cell line:NCI-H727,CAGE


In [60]:
features2 = []
for assay in assays:
    for norm_ord in [1, 2, np.inf]:
        f = f"{assay}_L{norm_ord}"
        V[f] = np.linalg.norm(df.loc[:, (metadata.assay==assay).values], ord=norm_ord, axis=1)
        features2.append(f)
V

,chrom,pos,ref,alt,label,ENCFF833POA,ENCFF110QGM,ENCFF880MKD,ENCFF463ZLQ,ENCFF890OGQ,...,DNASE_Linf,ATAC_L1,ATAC_L2,ATAC_Linf,CHIP_L1,CHIP_L2,CHIP_Linf,CAGE_L1,CAGE_L2,CAGE_Linf
0,1,3080038,T,C,True,0.020627,0.027453,0.022877,0.014595,0.011404,...,0.085219,0.457365,0.156471,0.085265,160.581253,3.416873,0.378062,9.354065,0.458088,0.080701
1,1,3774964,A,G,True,0.034373,0.026412,0.037801,0.026490,0.027976,...,0.685573,0.802904,0.307402,0.241692,323.179993,7.474385,1.229871,60.550163,3.681484,1.254374
2,1,6616131,C,T,False,0.013653,0.013146,0.022949,0.011773,0.012057,...,0.054407,0.368464,0.117766,0.047055,147.108810,3.247007,0.746749,23.772612,1.090832,0.124088
3,1,7665224,C,A,False,0.029997,0.034185,0.063735,0.015230,0.026252,...,0.063735,0.773678,0.249029,0.103679,226.499405,3.852591,0.199620,14.615899,0.776670,0.118764
4,1,8407293,G,A,False,0.020948,0.011623,0.015968,0.010985,0.022360,...,0.112507,0.422663,0.136117,0.056926,182.710037,3.876970,0.403559,22.886009,1.117955,0.161273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,22,47019717,G,T,False,0.010188,0.015055,0.031079,0.005793,0.014833,...,0.031079,0.513113,0.172076,0.097223,106.114189,1.913289,0.222389,15.310356,0.889503,0.168659
1779,22,47990921,C,T,True,0.042335,0.041975,0.008428,0.005667,0.005185,...,0.526509,0.242150,0.084729,0.043919,178.090881,3.635521,0.398041,5.149686,0.275685,0.047838
1780,22,50007172,T,C,False,0.030721,0.039224,0.036249,0.023633,0.029598,...,0.267857,0.796434,0.258318,0.119519,382.466522,7.878907,1.120741,17.865524,0.943486,0.213693
1781,22,50190508,G,A,True,0.185299,0.264034,0.273924,0.256359,0.342722,...,1.106204,4.570937,1.468735,0.641387,1186.245361,25.023426,1.829819,481.724548,21.132944,2.091627


In [61]:
res2 = []
for c in features2:
    res2.append([c, average_precision_score(V["label"], V[c])])
res2 = pd.DataFrame(res2, columns=["feature", "AUPRC"]).sort_values("AUPRC", ascending=False)
res2

,feature,AUPRC
1,DNASE_L2,0.740533
0,DNASE_L1,0.739937
11,CAGE_Linf,0.729963
2,DNASE_Linf,0.728030
10,CAGE_L2,0.715029
7,CHIP_L2,0.704423
3,ATAC_L1,0.702140
6,CHIP_L1,0.700948
4,ATAC_L2,0.700633
9,CAGE_L1,0.698352


In [59]:
res

,feature,AUPRC,name,file,clip,scale,sum_stat,description,assay,sample
0,ENCFF083VCD,0.734845,ENCFF083VCD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:lung female embryo (76 days),DNASE,lung female embryo (76 days)
1,ENCFF978RWQ,0.733138,ENCFF978RWQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:stomach female embryo (98 days),DNASE,stomach female embryo (98 days)
2,ENCFF082SFS,0.732975,ENCFF082SFS,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:heart male adult (27 years) and male adu...,DNASE,heart male adult (27 years) and male adult (35...
3,ENCFF130BQK,0.732596,ENCFF130BQK,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:femur female embryo (98 days),DNASE,femur female embryo (98 days)
4,ENCFF818FXA,0.731510,ENCFF818FXA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:hepatocyte,DNASE,hepatocyte
...,...,...,...,...,...,...,...,...,...,...
5308,ENCFF031GKU,0.523921,ENCFF031GKU,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:EP300:suprapubic skin female adult (51 year),CHIP,EP300:suprapubic skin female adult (51 year)
5309,ENCFF149ZYU,0.522652,ENCFF149ZYU,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H2BK15ac:H9,CHIP,H2BK15ac:H9
5310,ENCFF078WGV,0.522403,ENCFF078WGV,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:EP300:esophagus muscularis mucosa female ...,CHIP,EP300:esophagus muscularis mucosa female adult...
5311,ENCFF331CAB,0.517747,ENCFF331CAB,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:EP300:subcutaneous adipose tissue male ad...,CHIP,EP300:subcutaneous adipose tissue male adult (...
